# 다중 레이블 분류 (Multi-Label Classification)

다중 레이블 분류는 하나의 샘플이 여러 개의 클래스(레이블)에 동시에 속할 수 있는 분류 문제입니다.

## 예시
- 이메일 분류: 하나의 이메일이 "업무", "중요", "개인" 등 여러 카테고리에 동시에 속할 수 있음
- 영화 장르 분류: 하나의 영화가 "액션", "코미디", "로맨스" 등 여러 장르에 속할 수 있음
- 의료 진단: 환자가 여러 질병을 동시에 가질 수 있음


In [1]:
# 1. 다중 레이블 데이터 생성
import numpy as np
from sklearn.datasets import make_multilabel_classification

# 다중 레이블 데이터 생성
result = make_multilabel_classification(
    n_samples=100,      # 샘플 수
    n_features=20,      # 특성 수  
    n_classes=5,        # 가능한 총 클래스(레이블) 수
    n_labels=2,         # 각 샘플이 가질 수 있는 평균 레이블 수
    random_state=42
)

# 결과에서 X, y 추출 (첫 두 요소만 사용)
X, y = result[0], result[1]

print("X shape:", X.shape)
print("y shape:", y.shape)
print("y type:", type(y))
print("\n첫 5개 샘플의 레이블 (이진 행렬):")
print(y[:5])


X shape: (100, 20)
y shape: (100, 5)
y type: <class 'numpy.ndarray'>

첫 5개 샘플의 레이블 (이진 행렬):
[[0 0 0 1 0]
 [1 1 1 0 0]
 [0 0 1 1 0]
 [1 0 0 0 0]
 [1 0 1 0 0]]


In [2]:
# 2. 훈련 및 테스트 세트 분리
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print("훈련 세트 크기:")
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("\n테스트 세트 크기:")
print("X_test shape:", X_test.shape)  
print("y_test shape:", y_test.shape)


훈련 세트 크기:
X_train shape: (70, 20)
y_train shape: (70, 5)

테스트 세트 크기:
X_test shape: (30, 20)
y_test shape: (30, 5)


## MultiOutputClassifier와 SVM

**MultiOutputClassifier**는 각 출력(레이블)에 대해 독립적인 분류기를 훈련합니다.  
내부적으로 One-vs-Rest 전략과 유사하게 작동합니다.

- 각 레이블마다 별도의 이진 분류기를 생성
- 병렬 처리 가능 (n_jobs=-1)
- 확률 예측 지원 (probability=True)


In [3]:
# 3. MultiOutputClassifier와 SVM 모델 설정
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier

# 기본 SVM 분류기 설정
base_svm = SVC(
    kernel='linear',     # 선형 커널 사용
    probability=True,    # 확률 예측을 위해 설정
    random_state=42
)

# 다중 출력 분류기 설정
multi_label_svm = MultiOutputClassifier(
    base_svm,
    n_jobs=-1  # 병렬 처리 (가능한 경우)
)

print("모델 설정 완료")
print("기본 분류기:", type(base_svm).__name__)
print("다중 출력 분류기:", type(multi_label_svm).__name__)


모델 설정 완료
기본 분류기: SVC
다중 출력 분류기: MultiOutputClassifier


In [4]:
# 4. 모델 훈련
print("모델 훈련 시작...")
multi_label_svm.fit(X_train, y_train)
print("모델 훈련 완료!")


모델 훈련 시작...
모델 훈련 완료!


In [5]:
# 5. 예측
y_pred = multi_label_svm.predict(X_test)
y_pred_proba = multi_label_svm.predict_proba(X_test)  # 확률 예측

print("실제 레이블 (첫 5개):")
print(y_test[:5])
print("\n예측된 레이블 (첫 5개):")
print(y_pred[:5])

# 예측 확률 정보
print(f"\n확률 예측 결과:")
print(f"확률 배열 개수: {len(y_pred_proba)} (각 레이블별)")
print(f"첫 번째 레이블의 확률 shape: {y_pred_proba[0].shape}")


실제 레이블 (첫 5개):
[[0 0 0 0 1]
 [0 1 1 1 0]
 [1 1 0 0 1]
 [1 1 1 1 0]
 [0 1 0 0 0]]

예측된 레이블 (첫 5개):
[[0 0 0 1 0]
 [1 0 1 1 0]
 [1 1 0 0 1]
 [1 0 1 0 0]
 [0 1 0 1 0]]

확률 예측 결과:
확률 배열 개수: 5 (각 레이블별)
첫 번째 레이블의 확률 shape: (30, 2)


## 다중 레이블 분류 평가 지표

다중 레이블 분류에서는 일반적인 정확도와 다른 여러 평가 지표를 사용합니다:

1. **Hamming Loss**: 잘못 예측된 레이블의 비율 (0에 가까울수록 좋음)
2. **Jaccard Similarity Score**: 모든 레이블을 정확히 예측한 샘플의 비율 (엄격한 지표)
3. **F1-Score**: 
   - **Micro F1**: 전체 TP, FP, FN을 합산하여 계산 (레이블 불균형에 덜 민감)
   - **Macro F1**: 각 레이블에 대한 F1-score를 계산한 후 평균 (레이블 불균형에 민감)


In [6]:
# 6. 모델 평가
from sklearn.metrics import hamming_loss, jaccard_score, f1_score

# 1) 햄밍 손실 (Hamming Loss)
# 잘못 예측된 레이블의 비율. 0에 가까울수록 좋습니다.
h_loss = hamming_loss(y_test, y_pred)
print(f"Hamming Loss: {h_loss:.4f}")

# 2) Jaccard 유사도 점수 (정확 일치 비율)
# 모든 레이블을 정확히 예측한 샘플의 비율 (엄격한 지표)
# average='samples'는 각 샘플에 대해 정확히 일치하는 레이블의 비율을 계산한 후 평균
jaccard_similarity = jaccard_score(y_test, y_pred, average='samples')
print(f"Jaccard Similarity Score (Exact Match Ratio): {jaccard_similarity:.4f}")

# 3) F1-스코어
# micro: 전체 TP, FP, FN을 합산하여 계산 (레이블 불균형에 덜 민감)
f1_micro = f1_score(y_test, y_pred, average='micro')
print(f"Micro F1-score: {f1_micro:.4f}")

# macro: 각 레이블에 대한 F1-score를 계산한 후 평균 (레이블 불균형에 민감)
f1_macro = f1_score(y_test, y_pred, average='macro')
print(f"Macro F1-score: {f1_macro:.4f}")


Hamming Loss: 0.2933
Jaccard Similarity Score (Exact Match Ratio): 0.4750
Micro F1-score: 0.6452
Macro F1-score: 0.6007


In [7]:
# 7. 각 레이블에 대한 예측 확률 확인 (예시)
print("첫 번째 테스트 샘플에 대한 각 레이블의 예측 확률:")
print("실제 레이블:", y_test[0])
print("예측 레이블:", y_pred[0])
print()

for i, prob in enumerate(y_pred_proba):
    # prob[0]은 첫 번째 테스트 샘플에 대한 확률
    prob_no = prob[0][0]   # 클래스 0 (레이블 없음)의 확률
    prob_yes = prob[0][1]  # 클래스 1 (레이블 있음)의 확률
    print(f"Label {i}: {prob_no:.4f} (없음), {prob_yes:.4f} (있음)")
    
print(f"\n총 {len(y_pred_proba)}개의 레이블에 대해 독립적으로 예측이 수행되었습니다.")

첫 번째 테스트 샘플에 대한 각 레이블의 예측 확률:
실제 레이블: [0 0 0 0 1]
예측 레이블: [0 0 0 1 0]

Label 0: 0.8405 (없음), 0.1595 (있음)
Label 1: 0.8837 (없음), 0.1163 (있음)
Label 2: 0.7038 (없음), 0.2962 (있음)
Label 3: 0.5942 (없음), 0.4058 (있음)
Label 4: 0.7238 (없음), 0.2762 (있음)

총 5개의 레이블에 대해 독립적으로 예측이 수행되었습니다.
